Not sure how this is different from other freature projection notebooks - grj 01/02/20

In [ ]:
import json
import integrated_cell
from integrated_cell import model_utils, utils
import os
import numpy as np

model_dir = '/allen/aics/modeling/gregj/results/integrated_cell//test_cbvae_avg_inten/2019-09-09-09:46:26/'
parent_dir = '/allen/aics/modeling/gregj/results/integrated_cell/'
suffix = '_174160'


networks, dp, args = utils.load_network_from_dir(model_dir, parent_dir, suffix = suffix)
    
enc = networks['enc']
dec = networks['dec']

enc.train(False)
dec.train(False)

dp.data['test']['CellId'] = dp.csv_data['CellId'].values[dp.data['test']['inds']]
dp.data['train']['CellId'] = dp.csv_data['CellId'].values[dp.data['train']['inds']]


results_dir = '{}/results/kl_demo{}/'.format(model_dir, suffix)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
print("Results dir: {}".format(results_dir))

dp.image_parent = '/allen/aics/modeling/gregj/results/ipp/scp_19_04_10/'

In [ ]:
import aicsfeature.kitchen_sink
from skimage.measure import label
from scipy.ndimage.morphology import binary_fill_holes
import tqdm
from skimage.filters import gaussian, threshold_otsu

def im2feats(im, extra_features = ["io_intensity", "bright_spots", "intensity", "skeleton", "texture"]):
    im_cell, im_nuc, im_structures, im_structures_seg, seg_cell, seg_nuc = im_process(im)

    feats_out = aicsfeature.kitchen_sink.kitchen_sink(im_cell = im_cell, im_nuc = im_nuc, im_structures = im_structures, seg_cell = seg_cell, seg_nuc = seg_nuc, extra_features = extra_features)
    feats_out_seg = aicsfeature.kitchen_sink.kitchen_sink(im_cell = im_cell, im_nuc = im_nuc, im_structures = im_structures_seg, seg_cell = seg_cell, seg_nuc = seg_nuc, extra_features = extra_features)

    return feats_out, feats_out_seg
    
def im_process(im):
    seg_nuc = binary_fill_holes(find_main_obj(im[0][[2]] > 1E-2))
    seg_cell = binary_fill_holes(find_main_obj(im[0][[0]] > 1E-2))
    seg_cell[seg_nuc] = 1
 
    im_nuc = (im[0][[2]]*(255)).astype('uint16') * seg_nuc    
    im_cell = (im[0][[0]]*(255)).astype('uint16') * seg_cell
    
    im_structures = [(i[[1]]*(255)).astype('uint16') * seg_cell for i in im]
    
    im_structures_seg = list()
    
    for i, im_structure in enumerate(im_structures):
        im_blur = gaussian(im_structure, 1)

        im_pix = im_structure[im_cell>0]
        if np.all(im_pix==0):
            im_structures_seg.append(im_structure)
            continue
        
        im_structures_seg.append(im_structure * (im_blur > threshold_otsu(im_blur[im_cell>0])))
    
    return im_cell, im_nuc, im_structures, im_structures_seg, seg_cell, seg_nuc
    
def find_main_obj(im_seg):
    im_label = label(im_seg)
    
    obj_index = -1
    max_cell_obj_size = -1
    for i in range(1, np.max(im_label)+1):
        obj_size = np.sum(im_label == i)
        if obj_size > max_cell_obj_size:
            max_cell_obj_size = obj_size
            obj_index = i
        
    main_obj = im_label == obj_index
    
    return main_obj

import warnings
warnings.filterwarnings("ignore")

import time

In [ ]:
features_dir = '{}/results/features/test/'.format(parent_dir)
if not os.path.exists(features_dir):
    os.makedirs(features_dir)

ndat = dp.get_n_dat('test')
    
save_real_feat_paths = ['{}/feat_{}.pkl'.format(features_dir, i) for i in range(ndat)]



In [ ]:
import multiprocessing as mp

def save_real_feats(inputs, mode='test'):
    
    save_path, i = inputs
    
    if os.path.exists(save_path):
        return
    
    print('.', end='')
    
    im, classes, ref = dp.get_sample(mode, [i])
    im = im.numpy()
    ch_name = dp.label_names[classes[0]]
    
    ch_names = np.concatenate([['Membrane', ch_name, 'DNA']])
    
    feats = im2feats(im)
    
    with open(save_path, "wb") as f:
        pickle.dump([feats, ch_names], f)
    
    return
    
#open a multi-processing pool
pool = mp.Pool()

rows_real = [[save_path, i] for save_path, i in zip(save_real_feat_paths, range(ndat))]
results = pool.imap_unordered(save_real_feats, rows_real)

# Don't forget to close the pool
pool.close()    

print('Done.')

In [ ]:

#load the features

import pickle
import pandas as pd
import scipy.stats

feats_membrane = list()
feats_dna = list()
feats_structure = list()

for i, save_path in enumerate(save_real_feat_paths):
    if not os.path.exists(save_path):
        continue
    
    try:
        with open(save_path, 'rb') as f:
            feats_raw, channel_names = pickle.load(f)
    except:
        print('could not load {}'.format(save_path))

    f1 = pd.concat([pd.concat(fs) for fs in feats_raw[1][2:3]], axis=1)
    f2 = pd.concat([pd.concat(fs) for fs in feats_raw[0][2:3]], axis=1)

    c_out = {}
    for c in f2.columns:
        c_out[c] = c[0:3] + '_seg' + c[3:] 

    f2 = f2.rename(columns=c_out)

    feats_struct = pd.concat([f1,f2], axis=1)
       
    feats = {}
    feats["DNA"] = feats_raw[0][0]
    feats["Membrane"] = feats_raw[0][1]
    feats[channel_names[1]] = feats_struct
    
    for channel in channel_names:
        feats[channel]['index'] = i
        feats[channel]['structure'] = channel
        
        if channel == "Membrane":
            feats_membrane.append(feats[channel])
        elif channel == "DNA":
            feats_dna.append(feats[channel])
        else:
            feats_structure.append(feats[channel])

feats_membrane = pd.concat(feats_membrane)
feats_structure = pd.concat(feats_structure)
feats_dna = pd.concat(feats_dna)


keep_inds = feats_structure['index'].values

In [ ]:
pd.concat([feats_membrane, feats_dna, feats_structure], axis=1).to_csv('{}/feats_test.csv'.format(results_dir))

In [ ]:
import torch

embeddings_path = '{}/embeddings.pth'.format(results_dir)
# losses_path = '{}/losses.pth'.format(results_dir)
# klds_path = '{}/klds.pth'.format(results_dir)
dimension_variation_path = '{}/dimension_variation.pth'.format(results_dir)

if not os.path.exists(embeddings_path):
    embeddings = get_latent_embeddings(enc, dec, dp)
    torch.save(embeddings, embeddings_path)

else:
    embeddings = torch.load(embeddings_path)

save_klds_path = "{}/klds.pkl".format(results_dir)

#from elbo_demo.ipynb
with open(save_klds_path, 'rb') as f:
    klds_ref, ref_sorted_inds, klds_struct, struct_sorted_inds = pickle.load(f)
    
#sort embeddings by KLD
embeddings_ref = embeddings['test']['ref']['mu'][:, ref_sorted_inds]
embeddings_struct = embeddings['test']['struct']['mu'][:, struct_sorted_inds]

#use embeddings for which we have features
ref_keep_cols = klds_ref[ref_sorted_inds] > 0.05
struct_keep_cols = klds_struct[struct_sorted_inds] > 0.05

embeddings_ref = embeddings_ref[keep_inds]
embeddings_ref = embeddings_ref.numpy()[:, ref_keep_cols]

embeddings_struct = embeddings_struct[keep_inds]
embeddings_struct = embeddings_struct.numpy()[:, struct_keep_cols]

#at this point features and embeddings should be in cooresponding matrices

In [ ]:
from sklearn import linear_model
from sklearn import decomposition
from sklearn import cross_decomposition

f_memb = feats_membrane.drop(columns=['index', 'structure'])
f_dna = feats_dna.drop(columns=['index', 'structure'])

feats = pd.concat([f_memb, f_dna], axis=1)
feats[np.isnan(feats)] = 0

#zscore
feats_z = feats.apply(scipy.stats.zscore)
#drop the nan columns
feats_z = feats_z.drop(columns = feats_z.columns[np.all(np.isnan(feats_z),0)])

# feats['mem_to_dna_main_area_total'] = feats['memb_main_area_total']/feats['dna_main_area_total']
# feats['mem_to_dna_perimeter_total'] = feats['memb_main_perimeter_total']/feats['dna_main_perimeter_total']

# model_save_path = '{}/feats_model_lasso.pkl'.format(results_dir)

# if not os.path.exists(model_save_path):
    
#     model = linear_model.MultiTaskLassoCV()
#     model.fit(feats_z, scipy.stats.zscore(embeddings_ref))
    
#     with open(model_save_path, "wb") as f:
#         pickle.dump([model, feats_z], f)
        
# else:
#     with open(model_save_path, 'rb') as f:
#         model, feats_z = pickle.load(f)


In [ ]:
feats_z.columns

cell_feats = ['cell_volume',
                'cell_surface_area',
                'cell_shape_1st_axis_length',
                'cell_shape_2nd_axis_length',
                'cell_shape_sphericity',
                'cell_intensity_mean',
                'cell_intensity_median',
                'cell_intensity_sum',
                'cell_intensity_mode',
                'cell_intensity_max',
                'cell_intensity_std',
                'cell_intensity_entropy',
                'cell_skeleton_voxels_number',
                'cell_skeleton_nodes_number',
                'cell_skeleton_degree_mean',
                'cell_skeleton_edges_number',
                'cell_skeleton_deg0_prop',
                'cell_skeleton_deg1_prop',
                'cell_skeleton_deg3_prop',
                'cell_skeleton_deg4p_prop']

dna_feats = ['dna' + f[4:] for f in cell_feats]

feats_z = feats_z[cell_feats + dna_feats]

model_save_path = '{}/feats_model_lasso_short_features.pkl'.format(results_dir)

# if not os.path.exists(model_save_path):
    
#     model = linear_model.MultiTaskLassoCV()
#     model.fit(feats_z, embeddings_ref)

#     with open(model_save_path, "wb") as f:
#         pickle.dump([model, feats_z], f)
        
# else:
#     with open(model_save_path, 'rb') as f:
#         model, feats_z = pickle.load(f)

        
        

In [ ]:
model = linear_model.MultiTaskElasticNetCV()
model.fit(embeddings_ref, feats_z)

In [ ]:
#rename the columns to make them pretty


coef = model.coef_.T

renamed_columns = {}

for c_in in feats_z.columns:
    
    c = c_in
    
    if c[0:4] == 'cell':
        c = 'Membrane' + c[4:]
        
    if c[0:3] == 'dna':
        c = 'DNA' + c[3:]        
        
        
    c = c.replace('volume', 'area')
    c = c.replace('surface_area', 'perimeter') 
    c = c.replace('shape_', '')
    c = c.replace('1st_axis_length', 'major axis length')
    c = c.replace('2nd_axis_length', 'minor axis length')    
    
    
    c = c.replace('_', ' ')
   
    
    
    renamed_columns[c_in] = c
    
feats_z = feats_z.rename(columns = renamed_columns)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.imshow(coef)
plt.colorbar()

In [ ]:
c = feats_z.columns[6]
# c = c.replace('1st_axis_length', 'length')
c

In [ ]:
print(feats_z.shape)
print(model.coef_.shape)
np.sort(model.coef_[:, np.where(feats_z.columns == 'DNA area')[0]])

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('default')

embedding_dims_to_plot = [0,1]

coeff_scale = 7.5
text_offset = 0.1


n_top_coeffs = 10



coeff_norm = np.linalg.norm(coef[embedding_dims_to_plot, :], axis = 0)
coeff_norm_argsorted = np.argsort(coeff_norm)[::-1]

print_coeff = np.zeros(len(coeff_norm))
print_coeff[coeff_norm_argsorted[0:n_top_coeffs]] = 1

plt.figure(figsize = [20,20])
plt.scatter(embeddings_ref[:,embedding_dims_to_plot[0]], embeddings_ref[:,embedding_dims_to_plot[1]], s = 5)

for i, f in enumerate(feats_z.columns):
    
    x_coeff = coef[embedding_dims_to_plot[0], i] * coeff_scale
    y_coeff = coef[embedding_dims_to_plot[1], i] * coeff_scale
    
    plt.arrow(0, 0, x_coeff, y_coeff, head_width = 0.1, color='k', alpha = 0.5)
    
    if print_coeff[i]:
        plt.text(x_coeff+text_offset+0.005, y_coeff-0.005 + text_offset, f, alpha = 1, color = 'w')
        plt.text(x_coeff+ text_offset, y_coeff + text_offset, f, alpha = 1)

plt.axis('equal')
plt.show()


In [ ]:
from integrated_cell.utils.plots import scatter_im
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

train_or_test = 'test'

def get_image(train_or_test, im_id, channels_to_show = [1,1,1]):

    x, _ , _ = dp.get_sample(train_or_test, [im_id])

    im_out = model_utils.tensor2img(x)
    
    for i, ch in enumerate(channels_to_show):
        if not ch:
            im_out[:,:,i] = 0
    
    
    #do the alpha layer
    alpha = np.expand_dims(np.any(im_out > 0,2).astype(im_out.dtype), 2)

    im_final = np.concatenate([im_out, alpha], 2)
    
    color_transform = np.array([[1, 1, 0, 0], [0, 1, 1, 0], [1, 0, 1, 0], [0,0,0,1]])
    
    #fix this
    im_reshape = im_final.reshape([160*96, 4]).T
    
    im_recolored = np.matmul(color_transform, im_reshape).T 
#     im_recolored[im_recolored[:,3]>0] = 255
    
    im_out = im_recolored.reshape([160,96, 4]) / 1.25
    
    im_out[im_out > 1] = 1

    return im_out


plt.style.use('dark_background')

def myfunc(i):
    im = get_image(train_or_test, keep_inds[i], channels_to_show = [1,0,1])
    return im


coeff_scale = 7.5
text_offset = 0.05

n_top_coeffs = 10


figsize = [40, 40]
dims_to_plot = [0, 1]

coeff_norm = np.linalg.norm(coef[dims_to_plot, :], axis = 0)
coeff_norm_argsorted = np.argsort(coeff_norm)[::-1]

print_coeff = np.zeros(len(coeff_norm))
print_coeff[coeff_norm_argsorted[0:n_top_coeffs]] = 1

X = embeddings_ref

plot_dims = [[0, 1], [2,3], [4,5]]

# plot_dims = [[2,3]]

for dims_to_plot in plot_dims:

    plt.figure(figsize=figsize)
    scatter_im(X, myfunc, dims_to_plot = dims_to_plot, zoom = 0.01* figsize[0], inset=False, inset_width_and_height=0.17, inset_scatter_size = 50)

    for i, f in enumerate(feats_z.columns):
        
        x_coeff = coef[dims_to_plot[0], i] * coeff_scale
        y_coeff = coef[dims_to_plot[1], i] * coeff_scale

        if print_coeff[i]:
            plt.arrow(0, 0, x_coeff, y_coeff, width = 0.03, head_width = 0.06, color='w', zorder=10000, alpha = 0.5)
            plt.text(x_coeff + text_offset + 0.005, y_coeff + text_offset - 0.01, s = f, fontsize=24, color = 'k', zorder=50000)
            plt.text(x_coeff + text_offset, y_coeff + text_offset, s = f, fontsize=24, color = 'w', zorder=60000)


    plt.savefig('{}/shape_space_with_vector_{}_{}.png'.format(results_dir, dims_to_plot[0], dims_to_plot[1]), bbox_inches='tight', dpi=90)
    # matplotlib.image.imsave('{}/vector_shape_space.png'.format(results_dir))
    plt.show()



In [ ]:

feature_names = [
    'obj_area_total',
#     'main_convex_area_total',
    "obj_perimeter_total",
    'obj_equivalent_diameter_mean',
#     'main_major_axis_length_mean',
#     'main_minor_axis_length_mean',
#     'cos_main_orientation_mean',
#     'sin_main_orientation_mean',
    "obj_mean_intensity_std",
    "obj_num_objs",
    "main_perim_to_area",
    "main_aspect_ratio",
    
    ]

f_struct = feats_structure[feature_names]
rename_dict = {}
for c in f_struct.columns:
    rename_dict[c] = 'struct_' + c
f_struct = f_struct.rename(columns = rename_dict)

feats = f_struct

model = linear_model.MultiTaskElasticNetCV()
model.fit(scipy.stats.zscore(embeddings_struct), feats.apply(scipy.stats.zscore))

In [ ]:

plt.style.use('dark_background')

def myfunc(i):
    im = get_image(train_or_test, keep_inds[i], channels_to_show = [1,1,1])
    return im


coeff_scale = 5
dims_to_plot = [0,1]

X = embeddings_struct

plt.figure(figsize=(40,40))
scatter_im(X, myfunc, dims_to_plot = dims_to_plot, zoom = 0.5, inset=False, inset_width_and_height=0.17, inset_scatter_size = 50)


for i, f in enumerate(feats.columns):
    
    x_coeff = model.coef_[i, dims_to_plot[0]] * coeff_scale
    y_coeff = model.coef_[i, dims_to_plot[1]] * coeff_scale
    
    plt.arrow(0, 0, x_coeff, y_coeff, head_width = 0.05, color='w', zorder=10000)
    plt.text(x_coeff + text_offset, y_coeff + text_offset, f, zorder=10000)

plt.show()

In [ ]:
plt.scatter(embeddings_struct[:,0], embeddings_struct[:, 1])
plt.axis('equal')